In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Veriyi okuma (Kaggle'da dosya yolu genelde şöyledir, dosya ismini kontrol et)
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Verinin ilk 5 satırını görme
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# 1. Gereksiz olan 'customerID' sütununu siliyoruz (Tahmin için işe yaramaz, sadece kimlik bilgisi)
df.drop('customerID', axis=1, inplace=True)

# 2. 'TotalCharges' sütunu normalde para tutarıdır ama bazen sistem bunu yazı (object) olarak okur.
# Bunu zorla sayıya çeviriyoruz. (Hatalı olanları 'NaN' yani boş yapar)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 3. Veri setinde eksik (boş) değer var mı diye kontrol ediyoruz
print("Eksik Değer Sayıları:")
print(df.isnull().sum())

# 4. Hedefimiz olan 'Churn' (Müşteri Kaybı) durumuna bakalım. Kaç kişi gitmiş, kaç kişi kalmış?
print("\nChurn (Kayıp) Dağılımı:")
print(df['Churn'].value_counts())

Eksik Değer Sayıları:
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

Churn (Kayıp) Dağılımı:
Churn
No     5174
Yes    1869
Name: count, dtype: int64


In [4]:
# 1. Eksik olan o 11 satırı siliyoruz
df.dropna(inplace=True)

# 2. Hedefimiz olan 'Churn' (Terk etme) sütununu sayıya çeviriyoruz.
# Yes -> 1 (Terk etti), No -> 0 (Terk etmedi)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# 3. Geri kalan tüm yazı içeren sütunları (Cinsiyet, İnternet Tipi vb.) otomatik olarak 0 ve 1'e çeviriyoruz.
# Buna "One-Hot Encoding" denir.
df = pd.get_dummies(df)

# Son halini görelim. Artık tabloda hiç harf olmamalı, her şey sayı olmalı.
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,0,True,False,False,True,True,...,False,True,False,False,False,True,False,False,True,False
1,0,34,56.95,1889.50,0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,False,True
2,0,2,53.85,108.15,1,False,True,True,False,True,...,False,True,False,False,False,True,False,False,False,True
3,0,45,42.30,1840.75,0,False,True,True,False,True,...,False,False,True,False,True,False,True,False,False,False
4,0,2,70.70,151.65,1,True,False,True,False,True,...,False,True,False,False,False,True,False,False,True,False


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. X ve y ayrımı (X: Bilgiler, y: Sonuç)
X = df.drop('Churn', axis=1) # Churn hariç her şey X
y = df['Churn']              # Sadece Churn y

# 2. Eğitim ve Test olarak bölme (%20 test için ayırıyoruz)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Modeli Kurma (Random Forest kullanıyoruz)
model = RandomForestClassifier(random_state=42)

# 4. Modeli Eğitme (Veriyi öğreniyor)
model.fit(X_train, y_train)

# 5. Test etme (Ayırdığımız veriyi tahmin etmeye çalışıyor)
y_pred = model.predict(X_test)

# 6. Başarı Oranını Görme
basari = accuracy_score(y_test, y_pred)
print(f"Modelin Doğruluk Oranı (Accuracy): %{basari * 100:.2f}")

print("\nDetaylı Rapor:")
print(classification_report(y_test, y_pred))

Modelin Doğruluk Oranı (Accuracy): %77.75

Detaylı Rapor:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1033
           1       0.61      0.47      0.53       374

    accuracy                           0.78      1407
   macro avg       0.71      0.68      0.69      1407
weighted avg       0.76      0.78      0.77      1407

